In [ ]:
import cdsapi
c = cdsapi.Client()
print("CDS API is working!")


In [ ]:
#ERA5-Land post-processed daily statistics from 1950 to present
#2M_Temp_Minimum

import cdsapi

dataset = "derived-era5-land-daily-statistics" 
request = {
    "variable": ["2m_temperature"],
    "year": "2020",
    "month": "03",
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15"
    ],
    "daily_statistic": "daily_minimum",
    "time_zone": "utc+05:00",
    "frequency": "3_hourly",
    "format": "netcdf",
    "area": [37.5, 68.7, 6.7, 97.25]
}

client = cdsapi.Client()

# Specify the output file path
output_file = "C:/Users/subha/Desktop/ERA-5/ERA5_Land_India_March_2020.nc"

# Retrieve the data
client.retrieve(dataset, request, output_file)

print(f"Data successfully downloaded to {output_file}")

In [ ]:
import xarray as xr
import rasterio
from rasterio.transform import from_bounds
import os

# Paths
netcdf_file = "C:/Users/subha/Desktop/ERA-5/ERA5_Land_India_March_2020.nc"
output_dir = "C:/Users/subha/Desktop/ERA-5/Output_TIFFs"  # Directory to save the TIFF files

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load the NetCDF file using xarray
ds = xr.open_dataset(netcdf_file)

# Select the variable of interest (adjust variable name if needed)
variable = "t2m"  # Replace with the actual variable name in your NetCDF file

# Extract coordinate information
lon = ds["longitude"].values  # Longitudes
lat = ds["latitude"].values  # Latitudes
transform = from_bounds(
    west=lon.min(),
    south=lat.min(),
    east=lon.max(),
    north=lat.max(),
    width=len(lon),
    height=len(lat)
)

# Check temporal variable
time_var = "valid_time"  # Replace 'valid_time' with the correct variable name for time if needed

# Loop through each time step and save as GeoTIFF
for i, timestamp in enumerate(ds[time_var].values):
    # Extract the data for the current time step using 'valid_time' as the dimension
    data_array = ds[variable].isel(valid_time=i)
    
    # Output file path
    output_file = os.path.join(output_dir, f"{variable}_{str(timestamp)[:10]}.tif")
    
    # Write to GeoTIFF
    with rasterio.open(
        output_file,
        "w",
        driver="GTiff",
        height=data_array.shape[0],
        width=data_array.shape[1],
        count=1,  # Single band
        dtype=data_array.dtype,
        crs="EPSG:4326",  # WGS84 CRS
        transform=transform,
    ) as dst:
        dst.write(data_array.values, 1)  # Write the data
    
    print(f"Saved {output_file}")

# Close the dataset
ds.close()

print("\nAll time steps successfully saved as GeoTIFF files!")


In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt
from glob import glob

# Path to the directory where GeoTIFFs are stored
output_dir = "C:/Users/subha/Desktop/ERA-5/Output_TIFFs"

# Find all GeoTIFF files in the directory
tiff_files = glob(os.path.join(output_dir, "*.tif"))

# Number of GeoTIFF files
num_files = len(tiff_files)
print(f"Found {num_files} GeoTIFF files.")

# Create subplots to display all GeoTIFFs together
cols = 3  # Number of columns for the grid
rows = -(-num_files // cols)  # Calculate rows needed (ceil division)
fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 5))
axes = axes.flatten()

# Plot each GeoTIFF file
for i, tiff_file in enumerate(tiff_files):
    with rasterio.open(tiff_file) as src:
        # Read the data from the GeoTIFF
        data = src.read(1)  # Assuming it's single-band data
        
        # Plot the data
        ax = axes[i]
        img = ax.imshow(data, cmap="viridis", extent=src.bounds)
        
        # Set the title from the filename (without extension)
        title = os.path.basename(tiff_file).replace(".tif", "")
        ax.set_title(title, fontsize=10)
        ax.axis("off")
        
        # Add a colorbar
        cbar = fig.colorbar(img, ax=ax, orientation="vertical", shrink=0.7)
        cbar.set_label("Temperature (K)")  # Adjust label based on your data

# Remove unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout and display
plt.tight_layout()
plt.show()


In [ ]:
output_image = "C:/Users/subha/Desktop/ERA-5/All_TIFFs_Plot.png"  # Output PNG file

fig.savefig(output_image, dpi=900, bbox_inches="tight")  # Save with 900 DPI
plt.close(fig)

print(f"Layout saved as {output_image} at 900 DPI.")